# Coastal Zone

The Coastal Grid dataset provides a global tiling system for geospatial analytics in coastal areas.
It supports scalable data processing workflows by offering structured grids at varying zoom levels
(5, 6, 7) and buffer sizes (500m, 1000m, 2000m, 5000m, 10000m, 15000m).

Each tile contains information on intersecting countries, continents, and Sentinel-2 MGRS tiles
as nested JSON lists. The dataset is particularly suited for applications requiring global coastal
coverage, such as satellite-based coastal monitoring, spatial analytics, and large-scale data processing.

Key Features:
- Global coverage of the coastal zone, derived from OpenStreetMap's generalized coastline (2023-02).
- Precomputed intersections with countries, continents, and MGRS tiles.
- Designed for use in scalable geospatial workflows.

This dataset is structured as a STAC collection, with individual items for each zoom level and buffer
size combination. Users can filter items by the `zoom` and `buffer_size` fields in the STAC metadata.

Please consider the following citation when using this dataset:

Floris Reinier Calkoen, Arjen Pieter Luijendijk, Kilian Vos, Etiënne Kras, Fedor Baart,
Enabling coastal analytics at planetary scale, Environmental Modelling & Software, 2024,
106257, ISSN 1364-8152, https://doi.org/10.1016/j.envsoft.2024.106257.

In [ ]:
import os

import dask

dask.config.set({"dataframe.query-planning": False})

import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from ipyleaflet import Map, basemaps

storage_options = {"account_name": "coclico"}

### Connect to the CoCliCo STAC 

In [ ]:
coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
coastal_zone_collection = coclico_catalog.get_child("coastal-grid")
coastal_zone_collection

### The collection includes multiple buffer distances

In [ ]:
list(coastal_zone_collection.get_all_items())

In [ ]:
item = coastal_zone_collection.get_item("coastal_grid_z6_10000m")
item

## The data is stored here

In [ ]:
href = item.assets["data"].href
href

## Read it into GeoPandas

In [ ]:
import fsspec
with fsspec.open(href, mode="rb", **storage_options) as f:
    coastal_zone = gpd.read_parquet(f)

In [ ]:
print(coastal_zone.shape)
coastal_zone.head()

## Show the data on an interactive map

In [ ]:
coastal_zone.hvplot.polygons(
    geo=True,
    tiles="ESRI",
    color="coastal_grid:id",  
    cmap="Category10", 
    line_color="orange",  
    alpha=0.8,  
    width=600, 
    legend=False
)